In [3]:
import math

In [16]:
# Givens
d = 7.94e-3  # [m]
L = ...  # [m]
sin_theta = 1 / 150  # []
h0, hf = 10e-2, 2e-2  # [m], [m]
L_box, W_box = 32e-2, 26e-2  # [m], [m]
eps = 0.0015  # [m] (surface roughness of PVC pipe)
mu = 0.89 #  [Pa * s] (water at 25 degrees C)

# Constants
rho = 998  # [kg/m^3]
g = 9.81  # [m/s^2]

# Initial Conditions
V0 = L_box * W_box * h0  # [m^3]
A_tube = math.pi * d ** 2 / 4  # [m^2]
A_box = L_box * W_box  # [m^2]
V0 = A_box * (h0 + hf)  # [m^3]

# Final Conditions
VF = A_box * hf

In [5]:
# Experimental Results
results = {
    20: (3, 19),
    30: (3, 34),
    40: (4, 26),
    60: (4, 48)
}
results = {Li * 1e-2: 60 * mins + secs for Li, (mins, secs) in results.items()}

def errors(model) -> int:
    for Li, t_avg in results.items():
        t_pred = model(Li)
        print(f"{Li=}m {t_avg=}s {t_pred=}s sqr_error={(t_avg - t_pred) ** 2}")

In [6]:
# Model 1: Simple Bernoulli's Equation
# Ignores: losses, height difference, tube length (!!)

# (1 / (2 * A_tube ** 2)) * (V') ** 2 =  (g / A_box) * V
k = A_tube * (2 * g / A_box) ** 0.5
# dV/dt = -k * sqrt(V)
# sqrt(V) = -kt + C
C = 2 * V0 ** 0.5

def V(t):
    """Volume of tank as a function of time"""
    return (-k / 2 * t + C/2) ** 2

# find root of function
# VF = (-k / 2 * t + V0 ** 0.5) ** 2
# t = 2/k (sqrt(V0) - sqrt(VF))

def m1_t(L):
    """Time to drain as a funciton of tube length"""
    return 2/k * (V0 ** 0.5 - VF ** 0.5)  # [s]

In [30]:
def reynolds_number(v):
    return rho * v * d / mu

def haaland_equation(re):
    inv_sqrt_f = -1.8 * math.log10((eps / d / 3.7) ** 1.1 + 6.9 / re)
    return (1 / inv_sqrt_f) ** 2

def solve():
    Vi = V0
    Vdot_i = 0
    n = 0
    delta_t = 0.1  # [s]
    while True:
        n += 1
        Vi += Vdot_i * delta_t
        vi = Vi / A_tube

        print(Vi)

        re = reynolds_number(vi)
        f = haaland_equation(re)

        Vdot_i = -A_tube * (Vi / A_box * (0.2 / d * f / 2 / g + 0.5)) ** 0.5

        if abs((Vi - VF) / VF) < 0.001:
            print(n, n * delta_t, Vi)
            print("done")
            break
        elif n > 300 / delta_t:
            print("too many steps")
            break

solve()

0.009984000000000002
0.009982556444227663
0.009981112991039836
0.00997966964043652
0.009978226392417718
0.009976783246983431
0.009975340204133662
0.00997389726386841
0.00997245442618768
0.00997101169109147
0.009969569058579784
0.009968126528652624
0.00996668410130999
0.009965241776551883
0.009963799554378306
0.00996235743478926
0.009960915417784747
0.009959473503364768
0.009958031691529326
0.00995658998227842
0.009955148375612054
0.009953706871530228
0.009952265470032945
0.009950824171120205
0.00994938297479201
0.009947941881048363
0.009946500889889263
0.009945060001314714
0.009943619215324717
0.009942178531919274
0.009940737951098384
0.009939297472862051
0.009937857097210276
0.009936416824143061
0.009934976653660407
0.009933536585762315
0.009932096620448787
0.009930656757719826
0.009929216997575431
0.009927777340015606
0.009926337785040353
0.009924898332649671
0.009923458982843563
0.009922019735622031
0.009920580590985075
0.009919141548932699
0.009917702609464903
0.009916263772581688
